# DeepFM - no context

In [1]:
# Required modules

import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2023-04-20 09:27:53.318413: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-20 09:27:53.481218: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-20 09:27:54.154720: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-20 09:27:54.158429: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 09:27:57.323217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# Load the data

use_cols = ['user_id:token', 'item_id:token', 'rating:float']
data = pd.read_csv('./dataset/tripadvisor/tripadvisor.inter', usecols=use_cols)
data.head()

,user_id:token,item_id:token,rating:float
0,5C28F393B23BB894523AE7126A7AE445,219668,5
1,3FA27F6E8AC712A82C69C4EDD8B912CC,223860,5
2,B99CFBB5411EDC8881D13B7A4B313ADA,75680,5
3,3FA27F6E8AC712A82C69C4EDD8B912CC,224783,5
4,7CEFF5C32BA1F3B186E7838C7D3FE25E,222984,5


In [3]:
# Encoding the user_id column

user_encoder = LabelEncoder()
data['user_id:token'] = user_encoder.fit_transform(data['user_id:token'].values)

# Encoding the item_id column
item_encoder = LabelEncoder()
data['item_id:token'] = item_encoder.fit_transform(data['item_id:token'].values)

In [4]:
# Renaming columns

data.columns = ['user_id', 'item_id', 'rating']

In [5]:
# Split the data into train and test sets

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [6]:
# Define the number of unique users and movies

num_users = data['user_id'].nunique()
num_movies = data['item_id'].nunique()

# Define embedding size

embedding_size = 10

In [7]:
# Model definition

# Define the input shape
input_shape = (train_data.shape[1] - 1,)

# Define input layers
user_input = tf.keras.layers.Input(shape=(1,))
movie_input = tf.keras.layers.Input(shape=(1,))

# Define user embedding
user_embedding = tf.keras.layers.Embedding(num_users, embedding_size, input_length=1)(user_input)
user_embedding = tf.keras.layers.Flatten()(user_embedding)

# Define movie embedding
movie_embedding = tf.keras.layers.Embedding(num_movies, embedding_size, input_length=1)(movie_input)
movie_embedding = tf.keras.layers.Flatten()(movie_embedding)

# Concatenate user and movie embeddings
concat = tf.keras.layers.concatenate([user_embedding, movie_embedding])

# Define FM part
fm = tf.keras.layers.Dense(1, activation=None)(concat)

# Define DNN part
dnn = tf.keras.layers.Dense(64, activation='relu')(concat)
dnn = tf.keras.layers.Dense(32, activation='relu')(dnn)
dnn = tf.keras.layers.Dense(1, activation=None)(dnn)

# Concatenate FM and DNN parts
concat = tf.keras.layers.concatenate([fm, dnn])

# Define output layer
output = tf.keras.layers.Flatten()(concat)

# Define the model
model = tf.keras.models.Model(inputs=[user_input, movie_input], outputs=output)

In [8]:
# Compile the model

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])

In [9]:
# Fit the model

history = model.fit([train_data['user_id'], train_data['item_id']], 
                    train_data['rating'], 
                    validation_data=([test_data['user_id'], test_data['item_id']], test_data['rating']),
                    epochs=100, batch_size=64)

Epoch 1/100
178/178 [==============================] - 1s 2ms/step - loss: 17.4629 - mae: 4.0704 - val_loss: 17.0263 - val_mae: 4.0184
Epoch 2/100
178/178 [==============================] - 0s 1ms/step - loss: 15.6091 - mae: 3.8253 - val_loss: 14.4085 - val_mae: 3.6521
Epoch 3/100
178/178 [==============================] - 0s 2ms/step - loss: 12.3984 - mae: 3.3063 - val_loss: 10.9244 - val_mae: 3.0089
Epoch 4/100
178/178 [==============================] - 0s 2ms/step - loss: 9.6346 - mae: 2.6703 - val_loss: 9.2022 - val_mae: 2.5072
Epoch 5/100
178/178 [==============================] - 0s 2ms/step - loss: 8.8174 - mae: 2.4009 - val_loss: 8.8947 - val_mae: 2.4053
Epoch 6/100
178/178 [==============================] - 0s 2ms/step - loss: 8.6070 - mae: 2.3497 - val_loss: 8.7323 - val_mae: 2.3765
Epoch 7/100
178/178 [==============================] - 0s 2ms/step - loss: 8.4300 - mae: 2.3192 - val_loss: 8.5760 - val_mae: 2.3532
Epoch 8/100
178/178 [==============================] - 0s 2ms/s

178/178 [==============================] - 0s 2ms/step - loss: 0.7153 - mae: 0.6637 - val_loss: 1.4134 - val_mae: 0.9490
Epoch 63/100
178/178 [==============================] - 0s 2ms/step - loss: 0.7032 - mae: 0.6577 - val_loss: 1.4038 - val_mae: 0.9398
Epoch 64/100
178/178 [==============================] - 0s 2ms/step - loss: 0.6925 - mae: 0.6520 - val_loss: 1.3946 - val_mae: 0.9369
Epoch 65/100
178/178 [==============================] - 0s 2ms/step - loss: 0.6832 - mae: 0.6469 - val_loss: 1.3859 - val_mae: 0.9319
Epoch 66/100
178/178 [==============================] - 0s 2ms/step - loss: 0.6744 - mae: 0.6420 - val_loss: 1.3845 - val_mae: 0.9274
Epoch 67/100
178/178 [==============================] - 0s 2ms/step - loss: 0.6666 - mae: 0.6378 - val_loss: 1.3750 - val_mae: 0.9279
Epoch 68/100
178/178 [==============================] - 0s 2ms/step - loss: 0.6592 - mae: 0.6336 - val_loss: 1.3702 - val_mae: 0.9243
Epoch 69/100
178/178 [==============================] - 0s 2ms/step - loss:

In [10]:
# Evaluate the model

model.evaluate([test_data['user_id'], test_data['item_id']], test_data['rating'])

89/89 [==============================] - 0s 907us/step - loss: 1.4017 - mae: 0.9199


[1.4017395973205566, 0.9199087619781494]

In [11]:
# Plot the model

tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Cross validation

In [ ]:
# Cross validation

from sklearn.model_selection import KFold

kf = KFold(n_splits=10)

for train_index, valid_index in kf.split(train_data):
    train_set = train_data.iloc[train_index]
    valid_set = train_data.iloc[valid_index]
    
    model = tf.keras.models.Model(inputs=[user_input, movie_input], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])
    
    model.fit([train_set['user_id'], train_set['item_id']], 
              train_set['rating'], 
              validation_data=([valid_set['user_id'], valid_set['item_id']], valid_set['rating']),
              epochs=100, batch_size=16, verbose=0)
    print(model.evaluate([test_data['user_id'], test_data['item_id']], test_data['rating']))